In [1]:
import os
os.chdir('/home/smallyan/eval_agent')
print(f"Working directory: {os.getcwd()}")

Working directory: /home/smallyan/eval_agent


In [2]:
# First, let's explore the arithmetic_eval repository structure
import os

repo_path = '/net/scratch2/smallyan/arithmetic_eval'
print("Repository structure:")
for root, dirs, files in os.walk(repo_path):
    # Skip hidden directories and common non-essential dirs
    dirs[:] = [d for d in dirs if not d.startswith('.') and d not in ['__pycache__', '.git', 'node_modules']]
    level = root.replace(repo_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f'{subindent}{file}')

Repository structure:
arithmetic_eval/
  documentation.pdf
  CodeWalkthrough.md
  plan.md
  .gitignore
  LICENSE
  evaluation/
    self_matching.ipynb
    generalization_eval_summary.json
    generalization_eval.ipynb
    consistency_evaluation.json
    gt1_results.json
    gt3_results.json
    gt2_results.json
  data/
    word2vec/
      family.txt
      capital-common-countries.txt
      capital-world.txt
      gram2-opposite.txt
      gram5-present-participle.txt
      questions-phrases.txt
      questions-words.txt
      gram8-plural.txt
      gram1-adjective-to-adverb.txt
      gram7-past-tense.txt
      gram4-superlative.txt
      currency.txt
      city-in-state.txt
      gram3-comparative.txt
      gram9-plural-verbs.txt
      gram6-nationality-adjective.txt
    fvs/
      lowercase_first_letter.json
      lowercase_last_letter.json
      country-currency.txt
      product-company.json
      country-currency.json
      person-sport.txt
      next-capital-letter.txt
      person

In [3]:
# Let's just look at the top-level structure
import os

repo_path = '/net/scratch2/smallyan/arithmetic_eval'
print("Top-level files and directories:")
for item in sorted(os.listdir(repo_path)):
    full_path = os.path.join(repo_path, item)
    if os.path.isdir(full_path):
        print(f"📁 {item}/")
    else:
        print(f"📄 {item}")

Top-level files and directories:
📁 .git/
📄 .gitignore
📄 CodeWalkthrough.md
📄 LICENSE
📁 arithmetic_claude_2025-12-23_02-35-29/
📁 cache/
📁 data/
📄 documentation.pdf
📁 evaluation/
📁 figures/
📄 plan.md
📁 scripts/


In [4]:
# Let's read the plan.md file
with open(os.path.join(repo_path, 'plan.md'), 'r') as f:
    plan_content = f.read()
print(plan_content)

# Plan
## Objective
Show that concept and token induction heads can identify subspaces of Llama-2-7b activations with coherent semantic and surface-level structure, enabling more accurate parallelogram arithmetic (e.g., Athens – Greece + China = Beijing) than using raw hidden states.

## Hypothesis
1. Poor parallelogram arithmetic results on raw Llama-2-7b hidden states are due to interference from irrelevant information in model activations.
2. Word2vec arithmetic is only effective when performed in a semantic subspace of model activations, not on the full hidden state space.
3. Concept and token induction heads operate in subspaces that represent different facets of words (semantic vs. surface-level).

## Methodology
1. Build concept and token lenses by summing OV matrices (O(l,h)V(l,h)) from top-k concept/token induction heads identified in prior work, creating transformations LCk and LTk.
2. Extract word embeddings by passing single words (optionally with task-specific prefixes) th

In [5]:
# Now let's read the CodeWalkthrough.md file
with open(os.path.join(repo_path, 'CodeWalkthrough.md'), 'r') as f:
    code_walk_content = f.read()
print(code_walk_content)

# Vector Arithmetic in Concept and Token Subspaces
*Code and data for short paper at the NeurIPS 2025 Mechanistic Interpretability Workshop. See paper website [here](https://arithmetic.baulab.info).*

In this work, we use the weights of concept and token induction heads discovered in ["The Dual-Route Model of Induction"](https://dualroute.baulab.info/) to analyze word embeddings. We find that using these heads to "focus" on semantic information can make word2vec-style analogies like `Athens - Greece + China = Beijing` work out much more cleanly than they do using raw hidden states. Doing the same with token induction heads can help with more wordform-focused word2vec tasks, like `dance - dancing + coding = code`. 

# Data
We use two datasets in this work, which each have a number of tasks. 
1. `word2vec` - original data from [Mikolov et al. (2013)](https://arxiv.org/pdf/1301.3781)
2. `fvs` - function vector tasks from [Todd et al. (2024)](https://functions.baulab.info/)

# Scripts
- Ru

In [6]:
# Let's explore the scripts folder
scripts_path = os.path.join(repo_path, 'scripts')
print("Scripts folder contents:")
for item in sorted(os.listdir(scripts_path)):
    print(f"  {item}")

Scripts folder contents:
  __pycache__
  all_parallelograms.py
  parallelogram_analysis.ipynb
  parallelogram_ranks.py
  parallelogram_ranks.sh
  parallelograms.py


In [7]:
# Let's read the main parallelograms.py helper module first
with open(os.path.join(scripts_path, 'parallelograms.py'), 'r') as f:
    parallelograms_content = f.read()
print(parallelograms_content)

''' 
At a particular layer, take word2vec style parallelograms for:
    - sum of concept head OVs lm_head(OV(Tokyo_ell) - OV(Japan_ell) + OV(Germany_ell)) = Berlin?
    - sum of token head OVs (same thing)
    - sum of all the head OVs (same thing)
    - raw hidden states at last token position 

Summing all the head OVs is a nice comparison bc you get that concept signal still but it's surely drowned out
by all the other things heads could be reading from that hidden state + contributing to resid. at a given time.
'''
import os 
import torch 
import json 
import argparse
import matplotlib.pyplot as plt 
from nnsight import LanguageModel

def logit_lens(concept_vec, model):
    with torch.no_grad():
        return model.lm_head(model.model.norm(concept_vec.cuda())).softmax(dim=-1).detach().cpu() # vocab_size 

def print_logit_lens(probs, tokenizer, label=''):
    topprobs, idxs = torch.topk(probs, k=10)
    print(f'{label} logit lens\t', [(tokenizer.decode(t), round(p.item(), 3)) for t

In [8]:
# Let's read the all_parallelograms.py script
with open(os.path.join(scripts_path, 'all_parallelograms.py'), 'r') as f:
    all_parallelograms_content = f.read()
print(all_parallelograms_content)

''' 
take word2vec style parallelograms for:
    - sum of concept head OVs lm_head(OV(Tokyo_ell) - OV(Japan_ell) + OV(Germany_ell)) = Berlin?
    - sum of token head OVs (same thing)
    - sum of all the head OVs (same thing)
    - raw head activations 

do this for all tasks, layers, and concept/token/all/raw

# tasks for word2vec 
task_list = [
    'capital-common-countries', 'capital-world', 'currency',
    'city-in-state', 'family', 
    'gram1-adjective-to-adverb',
    'gram2-opposite', 'gram3-comparative', 'gram4-superlative',
    'gram5-present-participle', 'gram6-nationality-adjective',
    'gram7-past-tense', 'gram8-plural', 'gram9-plural-verbs'
]
'''
import os 
import argparse
from nnsight import LanguageModel
from parallelograms import get_neighbors, calculate_save_scores

def loop_for_task(this_task, task_name, model, subfolders, layers, concept_k, token_k, w_prefix, dataset):
    sep = ' ' if dataset == 'word2vec' else '\t'
    print(task_name, w_prefix, this_task[0].split

In [9]:
# Let's look at the data folder structure
data_path = os.path.join(repo_path, 'data')
print("Data folder contents:")
for item in sorted(os.listdir(data_path)):
    full_path = os.path.join(data_path, item)
    if os.path.isdir(full_path):
        print(f"📁 {item}/")
        for subitem in sorted(os.listdir(full_path)):
            print(f"    {subitem}")
    else:
        print(f"📄 {item}")

Data folder contents:
📁 fvs/
    antonym.json
    antonym.txt
    capitalize-first-letter.txt
    capitalize-last-letter.txt
    capitalize-second-letter.txt
    capitalize.json
    capitalize.txt
    capitalize_first_letter.json
    capitalize_last_letter.json
    capitalize_second_letter.json
    country-capital.json
    country-capital.txt
    country-currency.json
    country-currency.txt
    english-french.json
    english-french.txt
    english-german.json
    english-german.txt
    english-spanish.json
    english-spanish.txt
    landmark-country.json
    landmark-country.txt
    lowercase-first-letter.txt
    lowercase-last-letter.txt
    lowercase_first_letter.json
    lowercase_last_letter.json
    national-parks.txt
    national_parks.json
    next-capital-letter.txt
    next-item.txt
    next_capital_letter.json
    next_item.json
    park-country.json
    park-country.txt
    person-instrument.json
    person-instrument.txt
    person-occupation.json
    person-occupation.

In [10]:
# Let's look at an example data file
with open(os.path.join(data_path, 'word2vec', 'capital-common-countries.txt'), 'r') as f:
    content = f.read()
print("capital-common-countries.txt sample:")
print(content[:1000])

capital-common-countries.txt sample:
: capital-common-countries
Athens Greece Baghdad Iraq
Athens Greece Bangkok Thailand
Athens Greece Beijing China
Athens Greece Berlin Germany
Athens Greece Bern Switzerland
Athens Greece Cairo Egypt
Athens Greece Canberra Australia
Athens Greece Hanoi Vietnam
Athens Greece Havana Cuba
Athens Greece Helsinki Finland
Athens Greece Islamabad Pakistan
Athens Greece Kabul Afghanistan
Athens Greece London England
Athens Greece Madrid Spain
Athens Greece Moscow Russia
Athens Greece Oslo Norway
Athens Greece Ottawa Canada
Athens Greece Paris France
Athens Greece Rome Italy
Athens Greece Stockholm Sweden
Athens Greece Tehran Iran
Athens Greece Tokyo Japan
Baghdad Iraq Bangkok Thailand
Baghdad Iraq Beijing China
Baghdad Iraq Berlin Germany
Baghdad Iraq Bern Switzerland
Baghdad Iraq Cairo Egypt
Baghdad Iraq Canberra Australia
Baghdad Iraq Hanoi Vietnam
Baghdad Iraq Havana Cuba
Baghdad Iraq Helsinki Finland
Baghdad Iraq Islamabad Pakistan
Baghdad Iraq Kabul Afg

In [11]:
# Let's look at the cache folder to see the causal scores used for head selection
cache_path = os.path.join(repo_path, 'cache')
print("Cache folder contents:")
for root, dirs, files in os.walk(cache_path):
    dirs[:] = [d for d in dirs if not d.startswith('.') and d != '__pycache__']
    level = root.replace(cache_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    if level < 3:  # Only go 3 levels deep
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Limit files shown
            print(f'{subindent}{file}')
        if len(files) > 5:
            print(f'{subindent}... and {len(files) - 5} more files')

Cache folder contents:
cache/
  parallelograms/
    word2vec/
      with_prefix/
        all/
          gram9-plural-verbs/
          gram3-comparative/
          family/
          gram7-past-tense/
          capital-common-countries/
          gram4-superlative/
          currency/
          city-in-state/
          gram5-present-participle/
          gram8-plural/
          gram2-opposite/
          gram6-nationality-adjective/
          gram1-adjective-to-adverb/
          capital-world/
        token/
          capital-common-countries/
          gram4-superlative/
          gram5-present-participle/
          gram1-adjective-to-adverb/
          gram2-opposite/
          gram9-plural-verbs/
          gram3-comparative/
          gram6-nationality-adjective/
          currency/
          capital-world/
          family/
          gram7-past-tense/
          gram8-plural/
          city-in-state/
        raw/
          capital-common-countries/
          currency/
          gram2-op

In [12]:
# Let's read the concept and token head orderings from the causal scores
import json

llama2_cache = os.path.join(cache_path, 'causal_scores', 'Llama-2-7b-hf')

# Read concept heads scores
with open(os.path.join(llama2_cache, 'concept_copying_len30_n1024.json'), 'r') as f:
    concept_scores = json.load(f)

# Read token heads scores  
with open(os.path.join(llama2_cache, 'token_copying_len30_n1024.json'), 'r') as f:
    token_scores = json.load(f)

print("Concept heads sample (first 5):")
print(json.dumps(concept_scores[:5], indent=2))
print(f"\nTotal concept head entries: {len(concept_scores)}")

print("\nToken heads sample (first 5):")
print(json.dumps(token_scores[:5], indent=2))
print(f"\nTotal token head entries: {len(token_scores)}")

Concept heads sample (first 5):
[
  {
    "layer": 0,
    "head_idx": 0,
    "score": -2.1439045667648315e-06
  },
  {
    "layer": 0,
    "head_idx": 1,
    "score": 2.4493783712387085e-06
  },
  {
    "layer": 0,
    "head_idx": 2,
    "score": -3.991648554801941e-06
  },
  {
    "layer": 0,
    "head_idx": 3,
    "score": -0.000103730708360672
  },
  {
    "layer": 0,
    "head_idx": 4,
    "score": 1.955777406692505e-06
  }
]

Total concept head entries: 1024

Token heads sample (first 5):
[
  {
    "layer": 0,
    "head_idx": 0,
    "score": -9.103678166866302e-07
  },
  {
    "layer": 0,
    "head_idx": 1,
    "score": 2.6207417249679565e-06
  },
  {
    "layer": 0,
    "head_idx": 2,
    "score": 1.9473955035209656e-06
  },
  {
    "layer": 0,
    "head_idx": 3,
    "score": 6.356276571750641e-06
  },
  {
    "layer": 0,
    "head_idx": 4,
    "score": -2.60770320892334e-07
  }
]

Total token head entries: 1024


In [13]:
# Let's look at the existing cached results to understand what the expected outcomes are
result_sample_path = os.path.join(cache_path, 'parallelograms', 'word2vec', 'no_prefix', 'concept', 
                                  'capital-common-countries', 'layer20_results.json')

with open(result_sample_path, 'r') as f:
    result_sample = json.load(f)

print("Sample result (capital-common-countries, concept lens, layer 20):")
print(json.dumps(result_sample, indent=2)[:2000])

Sample result (capital-common-countries, concept lens, layer 20):
{
  "ll_acc": 0.17588932806324112,
  "nn_acc": 0.8952569169960475,
  "n": 506,
  "ll_panswers": [
    0.016871096566319466,
    0.27948102355003357,
    3.292822828640518e-13,
    0.005217921920120716,
    4.5330580178415403e-07,
    3.4592510046092774e-15,
    7.32978673416429e-14,
    3.175024696280837e-12,
    0.002804484451189637,
    0.9881963133811951,
    2.9099675884936005e-05,
    1.1899654964508954e-05,
    0.03534001484513283,
    0.07750292867422104,
    0.7845572829246521,
    1.2284732292755507e-05,
    3.3544814061897688e-12,
    0.0016035947483032942,
    0.003403317416086793,
    0.864624559879303,
    2.437255286480422e-09,
    0.06975482404232025,
    0.022049961611628532,
    1.566780694984074e-13,
    0.005722846835851669,
    1.541125698167889e-06,
    2.0890686980323293e-15,
    7.91494521562141e-15,
    2.954594208182115e-11,
    0.019948270171880722,
    0.8652606010437012,
    0.0001203395659103

In [14]:
# Now let's look at various results across layers and lenses for a key task
layers = [0, 4, 8, 12, 16, 20, 24, 28, 31]
orderings = ['raw', 'concept', 'token', 'all']

results_summary = {}

for ordering in orderings:
    results_summary[ordering] = {}
    for layer in layers:
        result_path = os.path.join(cache_path, 'parallelograms', 'word2vec', 'no_prefix', ordering,
                                   'capital-common-countries', f'layer{layer}_results.json')
        if os.path.exists(result_path):
            with open(result_path, 'r') as f:
                result = json.load(f)
            results_summary[ordering][layer] = result['nn_acc']

print("Nearest Neighbor Accuracy for capital-common-countries task:")
print("Layer", end="\t")
for layer in layers:
    print(layer, end="\t")
print()

for ordering in orderings:
    print(ordering, end="\t")
    for layer in layers:
        if layer in results_summary[ordering]:
            print(f"{results_summary[ordering][layer]:.3f}", end="\t")
        else:
            print("N/A", end="\t")
    print()

Nearest Neighbor Accuracy for capital-common-countries task:
Layer	0	4	8	12	16	20	24	28	31	
raw	0.000	0.024	0.051	0.091	0.172	0.158	0.059	0.000	0.010	
concept	0.000	0.237	0.334	0.607	0.883	0.895	0.866	0.623	0.225	
token	0.000	0.022	0.034	0.051	0.055	0.073	0.042	0.000	0.008	
all	0.000	0.055	0.095	0.126	0.198	0.182	0.101	0.010	0.030	


In [15]:
# Let's also look at the family and present-participle tasks to verify pattern
for task in ['family', 'gram5-present-participle']:
    print(f"\n{task}:")
    print("Layer", end="\t")
    for layer in layers:
        print(layer, end="\t")
    print()
    
    for ordering in orderings:
        print(ordering, end="\t")
        for layer in layers:
            result_path = os.path.join(cache_path, 'parallelograms', 'word2vec', 'no_prefix', ordering,
                                       task, f'layer{layer}_results.json')
            if os.path.exists(result_path):
                with open(result_path, 'r') as f:
                    result = json.load(f)
                print(f"{result['nn_acc']:.3f}", end="\t")
            else:
                print("N/A", end="\t")
        print()


family:
Layer	0	4	8	12	16	20	24	28	31	
raw	0.002	0.006	0.002	0.002	0.006	0.004	0.002	0.000	0.016	
concept	0.002	0.014	

0.018	0.030	0.032	0.069	0.045	0.028	0.047	
token	0.002	0.014	0.010	

0.008	0.020	0.024	0.012	0.006	0.018	
all	0.004	0.016	0.024	0.018	0.012	0.034	0.014	0.010	

0.045	

gram5-present-participle:
Layer	0	4	8	12	16	20	24	28	31	
raw	0.000	0.019	0.025	0.057	0.108	0.028	0.007	0.003	0.030	
concept	0.000	0.042	0.035	0.159	

0.248	0.099	0.029	0.012	0.048	
token	0.000	0.147	0.123	0.366	0.542	0.401	0.197	0.083	0.057	
all	0.000	0.055	0.066	0.162	

0.246	0.132	0.057	0.023	0.046	


# Replication: Vector Arithmetic in Concept and Token Subspaces

## Overview
This notebook replicates the experiment from the paper "Vector Arithmetic in Concept and Token Subspaces" (NeurIPS 2025 Mechanistic Interpretability Workshop).

### Key Hypothesis
1. Poor parallelogram arithmetic on raw Llama-2-7b hidden states is due to interference from irrelevant information
2. Word2vec arithmetic is only effective in a semantic subspace, not on full hidden states
3. Concept and token induction heads operate in different subspaces (semantic vs. surface-level)

### Methodology
1. Build lenses by summing OV matrices from top-k concept/token induction heads
2. Extract word embeddings through Llama-2-7b at layer ℓ, transformed using lens matrices
3. Test parallelogram arithmetic: a - b + b' and check if a' is nearest neighbor
4. Compare: raw, concept lens, token lens, and all heads

In [16]:
# Check if CUDA is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

CUDA available: True
CUDA device: NVIDIA A100 80GB PCIe
CUDA memory: 85.09 GB


In [17]:
# Import necessary libraries
import os
import json
import torch
import numpy as np
from typing import Dict, List, Tuple, Optional

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

Using device: cuda


In [18]:
# Install nnsight if needed
try:
    from nnsight import LanguageModel
    print("nnsight is already installed")
except ImportError:
    print("Installing nnsight...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'nnsight', '-q'])
    from nnsight import LanguageModel
    print("nnsight installed successfully")

nnsight is already installed


## 1. Load Model and Data

In [19]:
# Load the Llama-2-7b model
print("Loading Llama-2-7b model...")
model = LanguageModel('meta-llama/Llama-2-7b-hf', device_map='cuda', dispatch=True)
print(f"Model loaded successfully!")
print(f"Hidden size: {model.config.hidden_size}")
print(f"Number of layers: {model.config.num_hidden_layers}")
print(f"Number of attention heads: {model.config.num_attention_heads}")

Loading Llama-2-7b model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully!
Hidden size: 4096
Number of layers: 32
Number of attention heads: 32


In [20]:
# Load the head ordering scores from cache
repo_path = '/net/scratch2/smallyan/arithmetic_eval'
cache_path = os.path.join(repo_path, 'cache')

# Load concept and token head scores
llama2_cache = os.path.join(cache_path, 'causal_scores', 'Llama-2-7b-hf')

with open(os.path.join(llama2_cache, 'concept_copying_len30_n1024.json'), 'r') as f:
    concept_scores = json.load(f)

with open(os.path.join(llama2_cache, 'token_copying_len30_n1024.json'), 'r') as f:
    token_scores = json.load(f)

print(f"Loaded {len(concept_scores)} concept head scores")
print(f"Loaded {len(token_scores)} token head scores")

# Sort by score to get top-k heads
concept_sorted = sorted([(d['layer'], d['head_idx'], d['score']) for d in concept_scores], 
                        key=lambda t: t[2], reverse=True)
token_sorted = sorted([(d['layer'], d['head_idx'], d['score']) for d in token_scores], 
                      key=lambda t: t[2], reverse=True)

print(f"\nTop 5 concept heads: {concept_sorted[:5]}")
print(f"Top 5 token heads: {token_sorted[:5]}")

Loaded 1024 concept head scores
Loaded 1024 token head scores

Top 5 concept heads: [(14, 1, 0.0010720472782850266), (14, 9, 0.0003007464110851288), (11, 22, 0.0002677999436855316), (13, 23, 0.00024266168475151062), (9, 25, 0.0001970883458852768)]
Top 5 token heads: [(16, 19, 0.0021728137508034706), (11, 15, 0.0012158188037574291), (11, 2, 0.000996168702840805), (12, 26, 0.0009576366282999516), (8, 26, 0.0007893270812928677)]


In [21]:
# Load word2vec task data
data_path = os.path.join(repo_path, 'data', 'word2vec')

def load_task_data(task_name: str, dataset: str = 'word2vec') -> List[str]:
    """Load task data from file."""
    filepath = os.path.join(repo_path, 'data', dataset, f'{task_name}.txt')
    with open(filepath, 'r') as f:
        content = f.read()
    # Skip the header line and filter empty lines
    lines = [l for l in content.split('\n')[1:] if l.strip() != '']
    return lines

# Load a sample task
capital_task = load_task_data('capital-common-countries')
print(f"Loaded {len(capital_task)} examples from capital-common-countries")
print(f"Sample examples:")
for ex in capital_task[:3]:
    print(f"  {ex}")

Loaded 506 examples from capital-common-countries
Sample examples:
  Athens Greece Baghdad Iraq
  Athens Greece Bangkok Thailand
  Athens Greece Beijing China


## 2. Implement Core Functions

The key operations are:
1. **get_ov_sum**: Construct the lens matrix by summing OV matrices from top-k heads
2. **proj_onto_ov**: Project a word's hidden state through the lens matrix  
3. **get_parallelogram_scores**: Compute a - b + b' and check if a' is the nearest neighbor

In [22]:
def get_ov_sum(model, head_ordering: str = 'concept', k: int = 80, rank: int = 4096) -> Optional[torch.Tensor]:
    """
    Construct the lens matrix by summing OV matrices from top-k heads.
    
    Args:
        model: The language model
        head_ordering: 'concept', 'token', 'all', or 'raw'
        k: Number of top heads to use
        rank: Rank for low-rank approximation (4096 = full rank)
    
    Returns:
        OV sum matrix of shape (hidden_size, hidden_size) or None for 'raw'
    """
    head_dim = model.config.hidden_size // model.config.num_attention_heads
    
    if head_ordering == 'raw':
        return None
    elif head_ordering == 'all':
        # Use all heads
        to_sum = [(l, h) for l in range(model.config.num_hidden_layers) 
                  for h in range(model.config.num_attention_heads)]
    else:
        # Use top-k heads from concept or token ordering
        if head_ordering == 'concept':
            sorted_heads = concept_sorted
        else:  # token
            sorted_heads = token_sorted
        to_sum = [(l, h) for l, h, _ in sorted_heads][:k]
    
    # Compute the OV sum
    with torch.no_grad():
        ov_sum = torch.zeros((4096, 4096), device='cuda')
        
        for l, h in to_sum:
            # Get V projection weights: (out_features, in_features)
            # Select rows for this head: (head_dim, hidden_size)
            V = model.model.layers[l].self_attn.v_proj.weight[h * head_dim : (h+1) * head_dim]
            
            # Get O projection weights: (hidden_size, num_heads * head_dim)
            # Select columns for this head: (hidden_size, head_dim)
            O = model.model.layers[l].self_attn.o_proj.weight[:, h * head_dim : (h+1) * head_dim]
            
            # OV = O @ V: (hidden_size, hidden_size)
            ov_sum += torch.matmul(O, V)
        
        # Low-rank approximation if needed
        if rank < model.config.hidden_size:
            U, S, Vh = torch.linalg.svd(ov_sum)
            ov_sum = (U[:, :rank] * S[:rank]) @ Vh[:rank]
    
    return ov_sum

# Test the function
print("Testing get_ov_sum...")
ov_concept = get_ov_sum(model, 'concept', k=80)
print(f"Concept OV sum shape: {ov_concept.shape}")
print(f"Concept OV sum norm: {torch.norm(ov_concept).item():.4f}")

Testing get_ov_sum...
Concept OV sum shape: torch.Size([4096, 4096])


Concept OV sum norm: 123.5509


In [23]:
def get_word_representation(word: str, model, layer_idx: int, ov_sum: Optional[torch.Tensor] = None, 
                             prefix: str = '', offset: int = -1) -> torch.Tensor:
    """
    Get the representation of a word at a given layer, optionally transformed by OV sum.
    
    Args:
        word: The word to get representation for
        model: The language model
        layer_idx: Layer to extract representation from
        ov_sum: OV sum matrix for transformation (None for raw)
        prefix: Optional prefix to add before word
        offset: Token position to extract (-1 for last token)
    
    Returns:
        Word representation tensor of shape (hidden_size,)
    """
    # Add space in front of word to ensure proper tokenization
    text = prefix + word.strip()
    
    with torch.no_grad():
        with model.trace(text):
            # Get hidden state at specified layer
            state = model.model.layers[layer_idx].output[0].squeeze()[offset].detach().save()
    
    # Apply OV transformation if provided
    if ov_sum is not None:
        return torch.matmul(ov_sum, state)
    else:
        return state

# Test the function
print("Testing get_word_representation...")
test_rep = get_word_representation("Athens", model, layer_idx=20, ov_sum=ov_concept)
print(f"Word representation shape: {test_rep.shape}")
print(f"Word representation norm: {torch.norm(test_rep).item():.4f}")

Testing get_word_representation...


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Word representation shape: torch.Size([4096])
Word representation norm: 141.4668


In [24]:
def get_all_neighbors(task_lines: List[str], model, layer_idx: int, 
                      ov_sum: Optional[torch.Tensor] = None, 
                      prefix: str = '', sep: str = ' ') -> Dict[str, torch.Tensor]:
    """
    Get representations for all unique words in a task.
    
    Args:
        task_lines: List of task examples (e.g., "Athens Greece Baghdad Iraq")
        model: The language model
        layer_idx: Layer to extract representations from
        ov_sum: OV sum matrix for transformation
        prefix: Optional prefix for each word
        sep: Separator between words in task lines
    
    Returns:
        Dictionary mapping words to their representations
    """
    # Collect all unique words
    words = set()
    for line in task_lines:
        parts = line.split(sep)
        words.update(parts)
    
    # Get representations for each word
    neighbors = {}
    for word in words:
        neighbors[word] = get_word_representation(word, model, layer_idx, ov_sum, prefix)
    
    return neighbors

# Test with a small subset
print("Testing get_all_neighbors...")
small_task = capital_task[:5]
neighbors = get_all_neighbors(small_task, model, layer_idx=20, ov_sum=ov_concept)
print(f"Number of unique words: {len(neighbors)}")
print(f"Sample words: {list(neighbors.keys())[:5]}")

Testing get_all_neighbors...


Number of unique words: 12
Sample words: ['Germany', 'China', 'Switzerland', 'Greece', 'Bangkok']
